# Select top-N assets with good indicators
The objective is to calculate the standard indicators for a top N number of assets. Then use the indicators to decide on a potentially significant set of assets to consider for the portfolio. Thereafter, apply the MPT monte carlo algorithm to construct a weigted portfolio. 

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore') #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; 
    else set it to False to turn off debugging mode '''
debug = True

## Initialize the classes

In [6]:
import os
import sys
from datetime import datetime, date, timedelta
from pyspark.sql import functions as F

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
from mining.modules.assets.etp import dailyTopN as topN

''' restart initiate classes '''
if debug:
    import importlib
    topN = importlib.reload(topN)

__desc__ = "analyze crypto market capitalization time series data"
clsTopN = topN.WeightedPortfolio(desc=__desc__)
print("\nClass initialization and load complete!")

All functional DAILYTOPN-libraries in ETP-package of ASSETS-module imported successfully!
sparkNoSQLwls Class initialization complete
sparkNoSQLwls Class initialization complete
execSession Class initialization complete
performIndex Class initialization complete
execSession Class initialization complete
dailyTopN Class initialization complete

Class initialization and load complete!


## Select topN assets

### Read top N mcap assets from tip sql db

Set the following parameters to select the mcap data from the database
* ```_num_assets``` (integer) limits the number of asset count
* ```_mcap_val_lb```(decimal) limits the asset selection by mcap_value
* ```_date``` (datetime) selects assets with values for that day
* ```_table```(string) by default is 'warehouse.mcap_past' where mcap daily data is stored

Extends the ```utils/etl/load/sparkDBwls``` package to read the data from database table


### Construct a dict with selected assets
The dictionary serves as an input to the ```mining/modules/assets/etp/performIndex``` package to compuiting the index values.

In [7]:
_num_assets=23
_mcap_val_lb=100000000.0
_date=date(2022,1,27)
_table='warehouse.mcap_past'
_idx_types=['adx','rsi','mfi','macd']
kwargs={
    "TABLENAME" :_table,
    "ASSETCOUNT":_num_assets,
    "INDEXTYPELIST":_idx_types,
    "STANDARDIZE" :'minmax',
    "COLPREFIX" :'scaled',
    "WINLENGTH" :7,
    "WINUNIT" :'DAY',
    "PCALOWERBOUND":0.7,
}

_portfolio=clsTopN.select_top_assets(
    mcap_date=_date,
    mcap_value_lb=_mcap_val_lb,
    **kwargs,
)

_portfolio[:1]

select * from warehouse.mcap_past wmp where wmp.mcap_date = '2022-01-27T00:00:00' and wmp.mcap_value > 100000000.0 and wmp.price_simp_ror >= 0 order by wmp.mcap_value DESC limit 23 


(None,)

In [120]:
from pyspark.sql import functions as F

print(clsTopN.index.select(F.col('asset_name'),F.col('1-PC[0]'),F.col('PC[0]'))\
      .sort('1-PC[0]',ascending=False).show())

+----------+-------------------+--------------------+
|asset_name|            1-PC[0]|               PC[0]|
+----------+-------------------+--------------------+
|      klay|                1.0| -0.6045888817873777|
|      near|             0.9812| -0.5825933347596355|
|      wbtc|              0.963|  -0.561450544816531|
|       leo|             0.9382| -0.5324668453417014|
|       axs|             0.8193|-0.39362426617566615|
|       omi|             0.8017| -0.3730915277081158|
|      hbar| 0.7777000000000001| -0.3450835443179698|
|       icp|             0.7497| -0.3124437229671345|
|     matic|              0.718| -0.2754386955070266|
|       cro|             0.6737|-0.22368907753623424|
|       ftt| 0.6688000000000001|-0.21799395493009524|
|      avax|             0.5475|-0.07641518666796848|
|      mana|             0.5454|-0.07396411256794662|
|     steth|              0.395| 0.10158752118306011|
|       okb|0.34630000000000005| 0.15840871847124743|
|       btc|0.33140000000000

## Write portfolio to mongodb

In [121]:
kwargs={
    "DESTINDBNAME":'tip-portfolio',
    "COLLPREFIX" : 'cherry_picked',
    "UIDSLIST" : ['uuid'],
    "DATECOLNAME":'asset.mcap.date',
}
_saved_port = clsTopN.write_asset_picks_to_db(
    portfolio_data=_portfolio,
    cols_dict={},
    **kwargs,
)
_saved_port[:1]

[[{'mcap.db.fk': 53843,
   'uuid': '63976ea07cc473c589257bce',
   'asset.name': 'matic',
   'asset.mcap.date': '2022-01-27T00:00:00',
   'asset.mcap.value': 12212915722.4229,
   'index.adx': -0.39521888110019904,
   'index.sharp': 16.42596803177328,
   'index.rsi': 0.2913695270171328,
   'index.mfi': 0.2913695270171328,
   'index.beta': -0.7605901418779902,
   'index.scaled.beta': 0.1883,
   'index.scaled.adx': 0.0,
   'index.scaled.mfi': 0.2732,
   'index.scaled.rsi': 0.2732,
   'index.scaled.sharp': 0.1905,
   'index.pc.0': -0.2754386955070266,
   'index.pc.1': -0.3140925697767417,
   'index.scaled.pc.0': 0.282,
   'index.scaled.pc.1': 0.9453,
   'index.scaled.1-pc.0': 0.718,
   'index.scaled.1-pc.1': 0.05469999999999997,
   'audit.mod.by': 'NUWAN',
   'audit.mod.dt': '2023-04-04T18:17:22',
   'audit.mod.proc': 'dailyTopN-function <write_asset_picks_to_db>',
   '_id': ObjectId('642bf93240bd6d8d5b8ac22a')},
  {'mcap.db.fk': 380,
   'uuid': '6397be217cc473c58945e15c',
   'asset.name': 